In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 56.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup as soup

# Obtener la data de los vinos desde la página de la CAV/tienda
data = []
driver = webdriver.Chrome()

for x in range(50):
    url = f"https://cav.cl/tienda?q=&hPP=20&idx=products&p={x}&fR%5Bfamily.name%5D%5B0%5D=Vinos&is_v=1"
    driver.get(url)
    
    try:
        age_verification_popup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'modalAge'))
        )
        yes_button = age_verification_popup.find_element(By.XPATH, ".//button[contains(., 'SI')]")
        yes_button.click()
    except (NoSuchElementException, TimeoutException):
        pass

    page_soup = soup(driver.page_source, 'html.parser')
    
    info = []
    for article in page_soup.find_all("article"):
        id = article.attrs.get("data-id")
        sku = article.attrs.get("data-sku")
        name = article.attrs.get("data-name")
        brand = article.attrs.get("data-brand")
        category = article.attrs.get("data-category")
        variant = article.attrs.get("data-variant")
        ref = article.find("a").attrs.get("href")
        info.append([id, sku, name, brand, category, variant, ref])
    
    data.append(info)
    
    # Guardar datos en un archivo CSV después de cada iteración
    df_partial = pd.DataFrame(info, columns=['id', 'sku', 'name', 'brand', 'category', 'variant', 'ref'])
    df_partial.drop_duplicates("id", inplace=True)
    df_partial.to_csv(f"data//raw//vinos_parte_{x}.csv", sep=',', mode='w', index=False)

driver.quit()

# Reshape la data
reshaped_data = [item for sublist in data for item in sublist]

# Crear el DataFrame
df = pd.DataFrame(reshaped_data, columns=['id', 'sku', 'name', 'brand', 'category', 'variant', 'ref'])

# Eliminar los duplicados
df.drop_duplicates("id", inplace=True)

# Guardar CSV de los vinos encontrados
df.to_csv("data//raw//vinos.csv", sep=',', mode='w', index=False)

# Borrar los pack de vinos
df_filtered = df.drop(axis=0, index=df.index[:8])

# Completar la URL en la columna 'ref'
df_filtered['ref'] = 'https://cav.cl/' + df_filtered['ref'].astype(str)

# Guardar CSV solo vinos
df_filtered.to_csv("url_products.csv", sep=',', mode='w', index=False)


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /home/vscode/.cache/selenium/chrome/linux64/119.0.6045.105/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x55a7c33745e3 <unknown>
#1 0x55a7c30370b7 <unknown>
#2 0x55a7c306de55 <unknown>
#3 0x55a7c306ab81 <unknown>
#4 0x55a7c30b547f <unknown>
#5 0x55a7c30abcc3 <unknown>
#6 0x55a7c30770e4 <unknown>
#7 0x55a7c30780ae <unknown>
#8 0x55a7c333ace1 <unknown>
#9 0x55a7c333eb7e <unknown>
#10 0x55a7c33284b5 <unknown>
#11 0x55a7c333f7d6 <unknown>
#12 0x55a7c330bdbf <unknown>
#13 0x55a7c3362748 <unknown>
#14 0x55a7c3362917 <unknown>
#15 0x55a7c3373773 <unknown>
#16 0x7f7371217ea7 start_thread
